In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
model_n = 12
sparse = 0
time_steps = [83, 85]

# Input Files
if sparse:
    draws_filename = f"/Volumes/KINGSTON/M4R_usb/R/modular_long/nm{model_n}[{time_steps[0]}-{time_steps[1]}]_sparse_draws.csv"
else:
    draws_filename = f"/Volumes/KINGSTON/M4R_usb/R/modular_long/nm{model_n}[{time_steps[0]}-{time_steps[1]}]_draws.csv"

# Read and Process Draws
draws = pd.read_csv(draws_filename)

In [ ]:
indices = np.arange(len(draws))
fig, axs = plt.subplots(2, 1, figsize=(12, 5), facecolor="#f5ebe4", sharex=True)

# Subplot 1:
axs[0].plot(indices[:], draws["w1[454]"][:], color='blue', label="", alpha=0.7, lw=0.15)
axs[0].set_title(r"Trace Plot of $\omega^1$[454]")
axs[0].set_ylabel("Posterior Samples")
# axs[0].legend()

# Subplot 2:
axs[1].plot(indices[:], draws["w1[455]"][:], color='blue', label="", alpha=0.7, lw=0.15)
axs[1].set_title(r"Trace Plot of $\omega^1$[455]")
axs[1].set_ylabel("Posterior Samples")
# axs[0].legend()

fig.suptitle(r"Model 1 ($\boldsymbol{\phi}$ removed) Traceplot Selection")

plt.tight_layout()
plt.show()

---

In [ ]:
model_n = 21
sparse = 0
time_steps = [83, 85]

# Input Files
if sparse:
    draws_filename = f"/Volumes/KINGSTON/M4R_usb/R/modular_extra/nm{model_n}[{time_steps[0]}-{time_steps[1]}]_sparse_draws.csv"
else:
    draws_filename = f"/Volumes/KINGSTON/M4R_usb/R/modular_extra/nm{model_n}[{time_steps[0]}-{time_steps[1]}]_draws.csv"

# Read and Process Draws
draws = pd.read_csv(draws_filename)

In [ ]:
phi_draws = draws.loc[:, draws.columns.str.startswith("phi")]
y_pp = phi_draws + np.random.normal(loc=0, scale=1, size=phi_draws.shape)
quantiles = y_pp.quantile([0.025, 0.975])

In [ ]:
q = quantiles.T
q.index = np.arange(1, len(q) + 1)

In [ ]:
zone = 0
y = pd.read_csv("smaller_zone/zone_0/c3_ALMA_1966_06.csv", usecols=["coord_index", "time_step", "latitude_rank", "longitude_rank", "w10", "tp"])
y = y[(y["time_step"]>=time_steps[0])&(y["time_step"]<=time_steps[1])][["latitude_rank", "longitude_rank", "time_step", "w10", "tp"]]

zone_min_lat = min(y["latitude_rank"])
zone_min_lon = min(y["longitude_rank"])
y["latitude_rank"] = y["latitude_rank"] - zone_min_lat + 1
y["longitude_rank"] = y["longitude_rank"] - zone_min_lon + 1
num_lat = max(y["latitude_rank"])
y = y.sort_values(by=["longitude_rank", "latitude_rank"])
y["coord_index"] = (y["longitude_rank"]-1)*num_lat + y["latitude_rank"]
y = y.sort_values(by="coord_index")
y = y.set_index("coord_index")

In [ ]:
# Stack "w10" and "tp" into one column
stacked = pd.DataFrame(y[["w10", "tp"]].stack().reset_index(drop=True))
# Copy selected columns
normalized = y[["w10", "tp"]].copy()

# Normalize each column separately (z-score)
normalized = (normalized - normalized.mean()) / normalized.std()

# Stack and reset index
stacked = pd.DataFrame(normalized.stack().reset_index(drop=True), columns=["value"])
# Reset index from 1 to N
stacked.index = np.arange(1, len(stacked) + 1)
stacked[0.025] = q[0.025]
stacked[0.975] = q[0.975]

In [ ]:
stacked["in_interval"] = ((stacked["value"] >= stacked[0.025]) & 
                          (stacked["value"] <= stacked[0.975])).astype(int)

In [ ]:
stacked["in_interval"].mean()